extract domain from nidos
done

In [1]:
import json
example=json.load(open('nido_subset/AAbV||GBBW01007738:genome.json'))

In [13]:
example['results'][0].keys()

dict_keys(['id', 'sequence', 'md5', 'crossReferences', 'openReadingFrames'])

In [113]:
from typing import Generator,Tuple
def iter_match(genome_dict:dict)->Generator[Tuple[dict,dict],None,None]:
    for orf in genome_dict['results'][0]['openReadingFrames']:
        for match in orf['protein']['matches']:
            yield (orf,match)

## Important Block below↓

In [163]:
from glob import glob
o=[]
b=0
for gfile in glob('nido_subset/*:genome.json'):
    # print('\n\n##'+gfile.split('/')[1].replace(':genome.json',''))
    genome_name=gfile.split('/')[1].replace(':genome.json','')
    genome_dict=json.load(open(gfile))
    genome_length=len(genome_dict['results'][0]['sequence'])
    for orf,match in iter_match(genome_dict):
        orf_info=orf['start'],orf['end'],orf['strand']
    #     if orf_info[2]!='SENSE':
    #         b=1
    #         break
    # if b==1:
    #     break
        match_info=(match['signature']['accession'],
                f"{match['signature']['name']}:{match['signature']['description']}",
                match['signature']['signatureLibraryRelease']['library'])
        if match_info[2]=='PFAM': #'PROSITE_PROFILES'
            for loc in match['locations']:
                data=(loc['start'],
                    loc['end'],
                    loc['hmmStart'],
                    loc['hmmEnd'],
                    loc['evalue'])
                entry={}
                entry['genome_name']=genome_name
                entry['genome_length']=genome_length
                entry['domain_accession']=match_info[0]
                
                entry['strand']=orf_info[2]
                if entry['strand']=='SENSE':
                    entry['start']=orf_info[0]+data[0]*3
                    entry['end']=orf_info[0]+data[1]*3
                else:
                    entry['start']=orf_info[1]-data[1]*3
                    entry['end']=orf_info[1]-data[0]*3
                entry['hmmStart']=data[2]
                entry['hmmEnd']=data[3]
                entry['evalue']=data[4]
                entry['domain_annotation']=match_info[1]
                o.append(entry)
                # match['evalue']
                # print(orf_info,end='\t')
                # print(f'\t {match_info}',end='\t')
                # print(f'\t {data}')

In [164]:
import pandas as pd
o_df=pd.DataFrame(o)
o_df.to_csv('tmp.csv',index=False)

In [153]:
o_df

,genome_name,domain_accession,strand,start,end,hmmStart,hmmEnd,evalue,domain_annotation
0,ACoV-WA3607||MK472070,PF19217,SENSE,7840,8839,5,351,2.600000e-121,"CoV_NSP4_N:Coronavirus replicase NSP4, N-terminal"
1,ACoV-WA3607||MK472070,PF19211,SENSE,580,1189,1,204,4.300000e-76,"CoV_NSP2_N:Coronavirus replicase NSP2, N-terminal"
2,ACoV-WA3607||MK472070,PF08716,SENSE,10939,11185,1,83,1.100000e-30,CoV_NSP7:Coronavirus replicase NSP7
3,ACoV-WA3607||MK472070,PF08717,SENSE,11188,11761,1,195,2.900000e-83,CoV_NSP8:Coronavirus replicase NSP8
4,ACoV-WA3607||MK472070,PF01661,SENSE,4213,4522,1,116,6.100000e-11,Macro:Macro domain
...,...,...,...,...,...,...,...,...,...
2299,PRRSV-2||U87392,PF05410,SENSE,361,673,1,105,8.300000e-58,Peptidase_C31:Porcine arterivirus-type cystein...
2300,PRRSV-2||U87392,PF14756,SENSE,1798,2236,1,147,3.900000e-89,Pdase_C33_assoc:Peptidase_C33-associated domain
2301,PRRSV-2||U87392,PF01606,SENSE,12699,13332,1,214,1.200000e-108,Arteri_env:Arterivirus envelope protein
2302,PRRSV-2||U87392,PF00951,SENSE,13872,14379,4,178,6.800000e-68,Arteri_Gl:Arterivirus GL envelope glycoprotein


In [126]:
from glob import glob

_match=[]
'''
entry: genome-id, genome-begin, genome-end, strand, accession,hmm-start,hmm-end,evalue
accession: name:description
'''

        
for gfile in glob('nido_subset/*:genome.json'):
    print('\n\n##'+gfile.split('/')[1].replace(':genome.json',''))
    genome_dict=json.load(open(gfile))
    for orf in genome_dict['results'][0]['openReadingFrames']:
        print(orf['start'],orf['end'],orf['strand'])
        for match in orf['protein']['matches']:
            infos=(match['signature']['accession'],
                f"{match['signature']['name']}:{match['signature']['description']}",
                match['signature']['signatureLibraryRelease']['library'])
            if infos[0]=='PF00680':
                _match.append(match)
            if infos[2]=='PFAM': #'PROSITE_PROFILES'
                for loc in match['locations']:
                    data=(loc['start'],
                        loc['end'],
                        loc['hmmStart'],
                        loc['hmmEnd'],
                        loc['evalue'],
                        )#match['evalue']
                    # o.append()
                    print(f'\t {infos}',end='\t')
                    print(f'\t {data}')
            



##ACoV-WA3607||MK472070
26633 27295 SENSE
238 12549 SENSE
	 ('PF19217', 'CoV_NSP4_N:Coronavirus replicase NSP4, N-terminal', 'PFAM')		 (2534, 2867, 5, 351, 2.6e-121)
	 ('PF19211', 'CoV_NSP2_N:Coronavirus replicase NSP2, N-terminal', 'PFAM')		 (114, 317, 1, 204, 4.3e-76)
	 ('PF08716', 'CoV_NSP7:Coronavirus replicase NSP7', 'PFAM')		 (3567, 3649, 1, 83, 1.1e-30)
	 ('PF08717', 'CoV_NSP8:Coronavirus replicase NSP8', 'PFAM')		 (3650, 3841, 1, 195, 2.9e-83)
	 ('PF01661', 'Macro:Macro domain', 'PFAM')		 (1325, 1428, 1, 116, 6.1e-11)
	 ('PF09401', 'CoV_NSP10:Coronavirus RNA synthesis protein NSP10', 'PFAM')		 (3961, 4082, 1, 122, 6.1e-57)
	 ('PF19213', 'CoV_NSP6:Coronavirus replicase NSP6', 'PFAM')		 (3313, 3566, 1, 263, 8.1e-83)
	 ('PF16348', 'CoV_NSP4_C:Coronavirus replicase NSP4, C-terminal', 'PFAM')		 (2891, 2986, 1, 96, 1.2e-36)
	 ('PF08710', 'CoV_NSP9:Coronavirus replicase NSP9', 'PFAM')		 (3845, 3952, 1, 111, 3.7e-37)
	 ('PF19212', 'CoV_NSP2_C:Coronavirus replicase NSP2, C-terminal', 

In [123]:
_match[2]['locations'][0]['hmmEnd'],_match[2]['locations'][0]['hmmStart']

(441, 86)

In [108]:
411-783,86-441

(-372, -355)

In [99]:
match

{'signature': {'accession': 'PF02497',
  'name': 'Arteri_GP4',
  'description': 'Arterivirus glycoprotein',
  'signatureLibraryRelease': {'library': 'PFAM', 'version': '36.0'},
  'entry': {'accession': 'IPR003412',
   'name': 'Arteri_GP4',
   'description': 'Arterivirus GP4 envelope glycoprotein',
   'type': 'FAMILY',
   'goXRefs': [],
   'pathwayXRefs': []}},
 'locations': [{'start': 24,
   'end': 201,
   'hmmStart': 1,
   'hmmEnd': 178,
   'hmmLength': 178,
   'hmmBounds': 'COMPLETE',
   'evalue': 2.6e-97,
   'score': 335.6,
   'envelopeStart': 24,
   'envelopeEnd': 201,
   'postProcessed': True,
   'location-fragments': [{'start': 24,
     'end': 201,
     'dc-status': 'CONTINUOUS'}]}],
 'evalue': 2.3e-97,
 'score': 335.8,
 'model-ac': 'PF02497'}

In [96]:
e[0]['locations']

[{'start': 481,
  'end': 595,
  'hmmStart': 1,
  'hmmEnd': 113,
  'hmmLength': 116,
  'hmmBounds': 'N_TERMINAL_COMPLETE',
  'evalue': 1.1e-27,
  'score': 107.8,
  'envelopeStart': 481,
  'envelopeEnd': 598,
  'postProcessed': True,
  'location-fragments': [{'start': 481,
    'end': 595,
    'dc-status': 'CONTINUOUS'}]},
 {'start': 780,
  'end': 895,
  'hmmStart': 3,
  'hmmEnd': 116,
  'hmmLength': 116,
  'hmmBounds': 'C_TERMINAL_COMPLETE',
  'evalue': 1.3e-19,
  'score': 81.7,
  'envelopeStart': 778,
  'envelopeEnd': 895,
  'postProcessed': True,
  'location-fragments': [{'start': 780,
    'end': 895,
    'dc-status': 'CONTINUOUS'}]}]

In [92]:
match['locations']

[{'start': 24,
  'end': 201,
  'hmmStart': 1,
  'hmmEnd': 178,
  'hmmLength': 178,
  'hmmBounds': 'COMPLETE',
  'evalue': 2.6e-97,
  'score': 335.6,
  'envelopeStart': 24,
  'envelopeEnd': 201,
  'postProcessed': True,
  'location-fragments': [{'start': 24,
    'end': 201,
    'dc-status': 'CONTINUOUS'}]}]

In [77]:
match.keys()

dict_keys(['signature', 'locations', 'model-ac'])

In [84]:
orf['protein']['matches'][4]

{'signature': {'accession': 'PS51951',
  'name': 'COV_NSP9_SSRNA_BD',
  'description': 'Coronavirus Nsp9 single-stranded RNA (ssRNA)-binding domain profile.',
  'signatureLibraryRelease': {'library': 'PROSITE_PROFILES',
   'version': '2022_05'},
  'entry': {'accession': 'IPR014822',
   'name': 'NSP9_CoV',
   'description': 'Non-structural protein NSP9, coronavirus',
   'type': 'DOMAIN',
   'goXRefs': [],
   'pathwayXRefs': []}},
 'locations': [{'start': 3845,
   'end': 3952,
   'score': 48.588364,
   'alignment': 'NNEIIPGKLKQRVVKAEGDG--FSVEGKALFNNEGGKTFMYAFVADKPDLKFVKWEFDGG-CNTIELEPPCRFAVESPNGPIIKYLYFVRNLNTLRRGAVLGFIGATVRLQ',
   'location-fragments': [{'start': 3845,
     'end': 3952,
     'dc-status': 'CONTINUOUS'}]}],
 'model-ac': 'PS51951'}

In [86]:
orf['protein']['matches'][0]

{'signature': {'accession': 'PF02497',
  'name': 'Arteri_GP4',
  'description': 'Arterivirus glycoprotein',
  'signatureLibraryRelease': {'library': 'PFAM', 'version': '36.0'},
  'entry': {'accession': 'IPR003412',
   'name': 'Arteri_GP4',
   'description': 'Arterivirus GP4 envelope glycoprotein',
   'type': 'FAMILY',
   'goXRefs': [],
   'pathwayXRefs': []}},
 'locations': [{'start': 24,
   'end': 201,
   'hmmStart': 1,
   'hmmEnd': 178,
   'hmmLength': 178,
   'hmmBounds': 'COMPLETE',
   'evalue': 2.6e-97,
   'score': 335.6,
   'envelopeStart': 24,
   'envelopeEnd': 201,
   'postProcessed': True,
   'location-fragments': [{'start': 24,
     'end': 201,
     'dc-status': 'CONTINUOUS'}]}],
 'evalue': 2.3e-97,
 'score': 335.8,
 'model-ac': 'PF02497'}

In [70]:
orf['protein']['matches'][0]['locations'][0]

{'start': 2534,
 'end': 2867,
 'hmmStart': 5,
 'hmmEnd': 351,
 'hmmLength': 352,
 'hmmBounds': 'INCOMPLETE',
 'evalue': 2.6e-121,
 'score': 416.7,
 'envelopeStart': 2529,
 'envelopeEnd': 2868,
 'postProcessed': True,
 'location-fragments': [{'start': 2534,
   'end': 2867,
   'dc-status': 'CONTINUOUS'}]}

In [64]:
orf['protein']['matches'][0]['signature']

{'accession': 'PF19217',
 'name': 'CoV_NSP4_N',
 'description': 'Coronavirus replicase NSP4, N-terminal',
 'signatureLibraryRelease': {'library': 'PFAM', 'version': '36.0'},
 'entry': {'accession': 'IPR043612',
  'name': 'CoV_NSP4_N',
  'description': 'Non-structural protein NSP4, N-terminal, coronavirus',
  'type': 'DOMAIN',
  'goXRefs': [],
  'pathwayXRefs': []}}

(19347, 19955, 'SENSE')

('PF00089', 'Trypsin', 'PFAM')